In [1]:
import json
import os
import re
import time

import numpy as np
import pandas as pd

from collections import defaultdict
from geopy.geocoders import Nominatim

%matplotlib inline
pd.options.display.max_columns = 99
os.chdir("/Users/yenchenchou/Documents/RMDS_YC/RiskScore/RMDS_COVID19_riskgenerator")

In [2]:
df = pd.read_csv(
    "data/external/Core-USA-July2020-Release-CORE_POI-2020_06-2020-07-13/core_poi-part1.csv.gz",
    compression="gzip")

In [3]:
df.head(5)

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
0,sg:0026d587c9b145b58272cbac7bdea56a,NaN,Vallone A John DDS,NaN,NaN,Offices of Dentists,Offices of Dentists,621210.0,41.158062,-87.891611,35 Briarcliff Prof Ctr,Bourbonnais,IL,60914,US,1.815939e+10,NaN,NaN,NaN,NaN,NaN,2019-07
1,sg:0037c4dc85f642c6aeaaf318e41b1bf7,NaN,Mitchell Cosmetology College,NaN,NaN,"Colleges, Universities, and Professional Schools","Colleges, Universities, and Professional Schools",611310.0,33.243518,-86.816563,116 1st St S,Alabaster,AL,35007,US,1.205664e+10,"{ ""Mon"": [], ""Tue"": [[""12:00"", ""16:00""]], ""Wed...",NaN,NaN,NaN,NaN,2019-07
2,sg:0041bdf50f984826acd68ce19922ecd4,NaN,Sullivan Chris M Dr,NaN,NaN,"Other Professional, Scientific, and Technical ...",Veterinary Services,541940.0,32.153705,-91.715387,403 Eighth St,Winnsboro,LA,71295,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07
3,sg:005761d9681947629bf02a64cfd4aec2,NaN,U Pick Carlsbad Strawberry Co,NaN,NaN,Specialty Food Stores,Fruit and Vegetable Markets,445230.0,33.135632,-117.326437,1000 Cannon Rd,Carlsbad,CA,92008,US,NaN,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
4,sg:007236f9d77d4158bbd4d55f39d98e49,NaN,CJ's Auto Glass Repair,NaN,NaN,Lumber and Other Construction Materials Mercha...,Other Construction Material Merchant Wholesalers,423390.0,40.470182,-74.408106,190 State Route 18,East Brunswick,NJ,8816,US,1.732994e+10,"{ ""Mon"": [[""8:00"", ""20:00""]], ""Tue"": [[""8:00"",...",NaN,NaN,NaN,NaN,2019-07


In [4]:
df.shape

(1179095, 22)

In [5]:
def data_filter(df):
    
    filter_ls = ["Restaurants and Other Eating Places", "Grocery Stores"]
    #     df = df[df["top_category"].isin(filter_ls)].copy()
    
    df = df[df["region"] == "CA"].copy()
    df = df[["safegraph_place_id", 
            "latitude",
            "longitude",
            "city",
            "postal_code"]]
    df["community"] = df["postal_code"].apply(lambda x: mapping(x))
    df = df[df["community"].notnull()]
    
    return df


def mapping(x):
    with open("data/internal/zipcode_mapper.json") as file:
        mapper_dict = json.load(file)
    mapper_dict = {str(key):val for key, val in mapper_dict.items()}
    try: 
        return mapper_dict[str(x)]
    except:
        pass

In [6]:
now = time.time()
df_filter = data_filter(df)
later = time.time()
print(later - now)

41.280043601989746


In [7]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43462 entries, 31 to 1179087
Data columns (total 6 columns):
safegraph_place_id    43462 non-null object
latitude              43462 non-null float64
longitude             43462 non-null float64
city                  43462 non-null object
postal_code           43462 non-null int64
community             43462 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 2.3+ MB


In [2]:
df_poi = pd.read_csv("data/processed/RMDS_poi.csv")
df_poi.head(10)

,safegraph_place_id,latitude,longitude,city,postal_code,community
0,sg:026dc0f4e4a948fd98e905baa687c82b,33.833054,-118.353688,Torrance,90503,Torrance
1,sg:03d14e5e90f447998afb1b8f27db9ba2,34.038608,-118.251283,Los Angeles,90015,South Park
2,sg:056f4ad3213b4cbaadd3c85481966466,33.790104,-118.177383,Long Beach,90806,missing
3,sg:05a81a17c5624213ab5c0fe3a86799c1,34.237689,-118.556153,Northridge,91324,Northridge
4,sg:0790c0739bd24738b45452127b73f317,33.958626,-118.184623,Cudahy,90201,Cudahy
5,sg:09eb0b9d42784b148b48956de7fdbd0b,33.864753,-118.387954,Hermosa Beach,90254,Hermosa Beach
6,sg:0b7913671dcf42b58022cdf7c01b204b,34.017126,-118.497800,Santa Monica,90401,Santa Monica
7,sg:0bc238907fb349feb30f856b033d6b68,33.916725,-118.012510,La Mirada,90638,La Mirada
8,sg:1157b197264b43c993a01bcea407828e,34.403087,-118.409712,Canyon Country,91387,Santa Clarita
9,sg:158d9042ff8a4b0faed033d93454020e,33.878555,-118.013369,La Mirada,90638,La Mirada


In [3]:
df_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43462 entries, 0 to 43461
Data columns (total 6 columns):
safegraph_place_id    43462 non-null object
latitude              43462 non-null float64
longitude             43462 non-null float64
city                  43462 non-null object
postal_code           43462 non-null int64
community             43462 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 2.0+ MB


In [2]:
# df_area = pd.read_csv("data/processed/RMDS_poi_area_square_feet.csv")
# df_area.head(10)

,safegraph_place_id,area_square_feet
0,sg:000190647c574858930fbf286b4e8ab8,38193
1,sg:0084ec4c6dec4c5c93d39e4059a60839,39049
2,sg:00ed33a7131544aeb3a59fd8ea372c8c,5453
3,sg:00f6617f8e4b466db9980b5886245a45,3528
4,sg:022c7b01c98249f7b70e9a9b3a8612b9,4692
5,sg:024e6ff879a642baa9480de09989255e,574103
6,sg:02b38387310a4316b26309d922600837,5247
7,sg:035833a15f704fb5bb90786134c25815,1122
8,sg:0416156e60424acfbff6248d5f042c2a,1252
9,sg:0495a6c2700e47d5be99c5ae80382922,8653


In [4]:
# df_area.info()

In [5]:
df_open = pd.read_csv("data/processed/RMDS_open_hours.csv")
df_open.head(10)

,safegraph_place_id,weekday,open_hours
0,sg:005761d9681947629bf02a64cfd4aec2,Mon,9.0
1,sg:005761d9681947629bf02a64cfd4aec2,Tue,9.0
2,sg:005761d9681947629bf02a64cfd4aec2,Wed,9.0
3,sg:005761d9681947629bf02a64cfd4aec2,Thu,9.0
4,sg:005761d9681947629bf02a64cfd4aec2,Fri,9.0
5,sg:005761d9681947629bf02a64cfd4aec2,Sat,9.0
6,sg:005761d9681947629bf02a64cfd4aec2,Sun,9.0
7,sg:0080800153224074974597d48f0a801b,Mon,0.0
8,sg:0080800153224074974597d48f0a801b,Tue,0.0
9,sg:0080800153224074974597d48f0a801b,Wed,0.0


In [7]:
df_area_original = pd.read_csv("data/external/SafeGraphPlacesGeoSupplementSquareFeet.csv.gz", compression="gzip")
df_area_original.head(10)

,safegraph_place_id,location_name,polygon_class,is_synthetic,includes_parking_lot,iso_country_code,area_square_feet
0,sg:000190647c574858930fbf286b4e8ab8,Liberty Tax Service,SHARED_POLYGON,False,False,CA,38193
1,sg:001341fe7e794ab6bd65bb80759a1ac6,Old Firehouse School,OWNED_POLYGON,False,False,US,5495
2,sg:002921847f104c93a6df47bb831dedd2,Tenth Avenue Park,OWNED_POLYGON,False,NaN,US,388046
3,sg:002d2e9f047d47ca8661abb8939096cf,Hunter Village Wines & Liquors,SHARED_POLYGON,False,False,US,687
4,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,SHARED_POLYGON,False,False,US,10925
5,sg:0061c1d8e124406cbcc97e40eb1dd585,Jungle Rapids Family Fun Park,OWNED_POLYGON,False,True,US,35487
6,sg:00772ea8f43a4c43b3eacd202dccf706,Watermill Express,OWNED_POLYGON,True,NaN,US,9864
7,sg:007ae3ae555d414c89328abe51916e46,Patient Advocate Group,SHARED_POLYGON,True,NaN,US,13305
8,sg:0083eaf7f3674a8994b622a5f432d591,Brunswick Zone XL Blaine,SHARED_POLYGON,False,False,US,59331
9,sg:0084ec4c6dec4c5c93d39e4059a60839,Pak Mail,SHARED_POLYGON,False,False,CA,39049


In [8]:
df_area_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929331 entries, 0 to 5929330
Data columns (total 7 columns):
safegraph_place_id      object
location_name           object
polygon_class           object
is_synthetic            bool
includes_parking_lot    object
iso_country_code        object
area_square_feet        int64
dtypes: bool(1), int64(1), object(5)
memory usage: 277.1+ MB


In [9]:
df_merge = df_poi.merge(df_area_original, how = "left")

In [10]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43462 entries, 0 to 43461
Data columns (total 12 columns):
safegraph_place_id      43462 non-null object
latitude                43462 non-null float64
longitude               43462 non-null float64
city                    43462 non-null object
postal_code             43462 non-null int64
community               43462 non-null object
location_name           38297 non-null object
polygon_class           38297 non-null object
is_synthetic            38297 non-null object
includes_parking_lot    34515 non-null object
iso_country_code        38297 non-null object
area_square_feet        38297 non-null float64
dtypes: float64(3), int64(1), object(8)
memory usage: 4.3+ MB


In [6]:
df_merge = df_open.merge(df_poi, how = "left")

In [7]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 557928 entries, 0 to 557927
Data columns (total 8 columns):
safegraph_place_id    557928 non-null object
weekday               557928 non-null object
open_hours            557928 non-null float64
latitude              152327 non-null float64
longitude             152327 non-null float64
city                  152327 non-null object
postal_code           152327 non-null float64
community             152327 non-null object
dtypes: float64(4), object(4)
memory usage: 38.3+ MB


In [8]:
df_merge.head()

,safegraph_place_id,weekday,open_hours,latitude,longitude,city,postal_code,community
0,sg:005761d9681947629bf02a64cfd4aec2,Mon,9.0,NaN,NaN,NaN,NaN,NaN
1,sg:005761d9681947629bf02a64cfd4aec2,Tue,9.0,NaN,NaN,NaN,NaN,NaN
2,sg:005761d9681947629bf02a64cfd4aec2,Wed,9.0,NaN,NaN,NaN,NaN,NaN
3,sg:005761d9681947629bf02a64cfd4aec2,Thu,9.0,NaN,NaN,NaN,NaN,NaN
4,sg:005761d9681947629bf02a64cfd4aec2,Fri,9.0,NaN,NaN,NaN,NaN,NaN
